# Model Predictions and Experimentation

In [1]:
import os
import numpy as np
# from roboflow import Roboflow
from ultralytics import YOLO
import torch
import geopandas as gpd
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
# from split_images import *
import rasterio as rio
import glob
from osgeo import gdal
import time

## Conducting inference / predictions on North Carolina Phase 3 SVF with Best SVF Model trained in New Mexico

A quick check of the current working directory below.

In [2]:
os.getcwd()

'/Users/rdzur/Documents/TEM/NC_P3_2015_Coleridge_SE'

Set the parameters for the model location on the filesystem, the source input images for running predictions.  Since this inference task is GPU intensive, the device setting is also important depending on the hardware platform, here, device='mps', refers to [Metal Performance Shaders framework](https://developer.apple.com/documentation/metalperformanceshaders) on Apple Silicon GPU systems. For other systems, modify the device settings to device='0,1', for example. The project parameter establishes where the detection results are saved.

In [3]:
model = YOLO("models/svf_best.pt")
results_prj = model.predict(source="SVF_2x2_Multiprocess",conf=0.5,stream=True,save=True,verbose=False,device='mps',project="runs/detect")

The code below verifies mps support.  See [Accelerated PyTorch training on Mac](https://developer.apple.com/metal/pytorch/).  


In [4]:
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


The cell below runs the building error/issue predictions and takes as input the 2x2 retiled SVF images, set below in the path variable.  The object detection labeled boxes are recorded.  

In [5]:
start_time = time.time()

cls_vals = {"0":"BLD_class_error"} #Build a dictionary using the data.yaml where the keys start from zero and values are the class name
geom = []
cls = []
conf = []
files = []
path = f"{os.getcwd()}/SVF_2x2_Multiprocess"
img_files = sorted(os.listdir(path))
for result,file in zip(results_prj,img_files):
    boxes = result.boxes
    with rio.open(f"{path}/{file}") as raster:   
        for box in boxes:
            xmin,ymin,xmax,ymax = box.xyxy[0]
            xmin,ymin,xmax,ymax = float(xmin),float(ymin),float(xmax),float(ymax)
            
            x1,y1 = rio.transform.xy(raster.transform,ymin,xmin)
            x1,y2 = rio.transform.xy(raster.transform,ymax,xmin)
            x2,y2 = rio.transform.xy(raster.transform,ymax,xmax)
            x2,y1 = rio.transform.xy(raster.transform,ymin,xmax)

            geom.append(Polygon([(x1,y1),(x1,y2),(x2,y2),(x2,y1)]))
            cls.append(cls_vals[str(int(box.cls[0]))])
            conf.append(float(box.conf))
            files.append(file.split(".")[0])

print(f"\n\n--- {time.time() - start_time} seconds ---")

Results saved to runs/detect/predict


--- 12.735567808151245 seconds ---


After running the YOLO building prediction model above, saved predictions are converted below to an output geospatial polygon format with coordinate reference information and saved an output folder and filename with can be modified as needed below.

In [6]:
gdf = gpd.GeoDataFrame({"geometry":geom, "conf":conf, "class":cls, "file":files},crs="EPSG:6543")
gdf.to_file("Geopredictions/building_issue_NC_P3_TEM_Coleridge_SE_SVF_NM_model.shp",driver="ESRI Shapefile")

The output polygon records prediction attribution such as:

- confidence,
- class, and
- file

The prediction can be reviewed in the runs/detection folder to view the SVF image with annotation: D03_37_10864903_20160228_SVF_R10_D16_8bit_1.tif 